In [81]:
import pandas as pd
runways = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/runways.csv')
regions = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/regions.csv')
navaids = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/navaids.csv')
countries = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/countries.csv')
airports = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/airports.csv')
airport_frequencies = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/airport-frequencies.csv')
airport_comments = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/airport-comments.csv')
regions

,id,code,local_code,name,continent,iso_country,wikipedia_link,keywords
0,302811,AD-02,02,Canillo Parish,EU,AD,https://en.wikipedia.org/wiki/Canillo,Airports in Canillo Parish
1,302812,AD-03,03,Encamp Parish,EU,AD,https://en.wikipedia.org/wiki/Encamp,Airports in Encamp Parish
2,302813,AD-04,04,La Massana Parish,EU,AD,https://en.wikipedia.org/wiki/La_Massana,Airports in La Massana Parish
3,302814,AD-05,05,Ordino Parish,EU,AD,https://en.wikipedia.org/wiki/Ordino,Airports in Ordino Parish
4,302815,AD-06,06,Sant Julià de Lòria Parish,EU,AD,https://en.wikipedia.org/wiki/Sant_Julià_de_Lòria,Airports in Sant Julià de Lòria Parish
...,...,...,...,...,...,...,...,...
3896,306317,ZW-MS,MS,Matabeleland South Province,AF,ZW,https://en.wikipedia.org/wiki/Matabeleland_Sou...,Airports in Matabeleland South Province
3897,306318,ZW-MV,MV,Masvingo Province,AF,ZW,https://en.wikipedia.org/wiki/Masvingo_Province,Airports in Masvingo Province
3898,306319,ZW-MW,MW,Mashonaland West Province,AF,ZW,https://en.wikipedia.org/wiki/Mashonaland_West...,Airports in Mashonaland West Province
3899,306320,ZW-U-A,U-A,(unassigned),AF,ZW,NaN,Airports in (unassigned)


## 1. How many airports, airfields and heliports exist in each country and continent?

### airports and heliports in each country, each airport has its own airfield. 

In [47]:
import country_converter as coco
wf = airports[airports["type"].str.contains("closed") == False]
wf = wf[wf["type"].str.contains("seaplane_base") == False]
wf = wf.replace(to_replace=["large_airport", "medium_airport",'small_airport'],value="airport")
wf = wf.groupby('type')['iso_country'].apply(lambda x: x.value_counts()).reset_index()
wf = wf.rename(columns = {'level_1':'country_code','iso_country':'count'})
wf['country_name'] = wf.country_code.apply(lambda x: coco.convert(names=x, to='name_short', not_found=None))
wf.sort_values('country_code')


,type,country_code,count,country_name
368,heliport,AD,2,Andorra
261,heliport,AE,200,United Arab Emirates
96,airport,AE,41,United Arab Emirates
319,heliport,AF,8,Afghanistan
70,airport,AF,67,Afghanistan
...,...,...,...,...
12,airport,ZA,571,South Africa
42,airport,ZM,117,Zambia
351,heliport,ZM,4,Zambia
340,heliport,ZW,5,Zimbabwe


### airports and heliports in each continent, each airport has its own airfield. Oceania is a continent?

In [70]:
import pycountry_convert as pc
wf = airports[airports["type"].str.contains("closed") == False]
wf = wf[wf["type"].str.contains("seaplane_base") == False]
wf = wf.replace(to_replace=["large_airport", "medium_airport",'small_airport','balloonport'],value="airport")
wf['continent'] = wf['continent'].fillna('NA')
wf = wf.groupby('type')['continent'].apply(lambda x: x.value_counts()).reset_index()
wf = wf.rename(columns = {'level_1':'continent_code','continent':'count'})
wf['continent_name'] = [pc.convert_continent_code_to_continent_name(x) for x in wf['continent_code']]
wf.sort_values('continent_code')



,type,continent_code,count,continent_name
4,airport,AF,3572,Africa
12,heliport,AF,209,Africa
6,airport,AN,30,Antarctica
13,heliport,AN,9,Antarctica
3,airport,AS,4023,Asia
8,heliport,AS,5712,Asia
2,airport,EU,6718,Europe
10,heliport,EU,1714,Europe
0,airport,NA,19421,North America
7,heliport,NA,8804,North America


## 2. What is the average elevation of the airports, airfields and heliports in each country?

In [79]:
import country_converter as coco
wf = airports[airports["type"].str.contains("closed") == False]
wf = wf[wf["type"].str.contains("seaplane_base") == False]
wf = wf.replace(to_replace=["large_airport", "medium_airport",'small_airport'],value="airport")
wf = wf.groupby([wf.iso_country, wf.type])['elevation_ft'].mean().round(2)
wf = pd.DataFrame(wf).reset_index()
wf = wf.rename(columns = {'iso_country':'country_code','elevation_ft':'average_elevation_ft'})
wf['country_name'] = wf.country_code.apply(lambda x: coco.convert(names=x, to='name_short', not_found=None))


wf

,country_code,type,average_elevation_ft,country_name
0,AD,heliport,3450.00,Andorra
1,AE,airport,124.97,United Arab Emirates
2,AE,heliport,113.79,United Arab Emirates
3,AF,airport,5028.52,Afghanistan
4,AF,heliport,3198.33,Afghanistan
...,...,...,...,...
404,ZA,heliport,3918.43,South Africa
405,ZM,airport,3570.41,Zambia
406,ZM,heliport,2658.50,Zambia
407,ZW,airport,3037.72,Zimbabwe


## 3. What is the estimated population of each country?

In [83]:
df = pd.read_csv('/workspace/Airport_Pipeline/Geonames_data/allCountries.txt', delimiter='\t', header=None)

# Display the first few rows of the DataFrame
df.head()

/tmp/ipykernel_958/721114556.py:1: DtypeWarning: Columns (9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/workspace/Airport_Pipeline/Geonames_data/allCountries.txt', delimiter='\t', header=None)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,2986043,Pic de Font Blanca,Pic de Font Blanca,"Pic de Font Blanca,Pic du Port",42.64991,1.53335,T,PK,AD,NaN,00,NaN,NaN,NaN,0,NaN,2860,Europe/Andorra,2014-11-05
1,2994701,Roc Mélé,Roc Mele,"Roc Mele,Roc Meler,Roc Mélé",42.58765,1.74028,T,MT,AD,"AD,FR",00,NaN,NaN,NaN,0,NaN,2803,Europe/Andorra,2020-06-10
2,3007683,Pic des Langounelles,Pic des Langounelles,Pic des Langounelles,42.61203,1.47364,T,PK,AD,"AD,FR",00,NaN,NaN,NaN,0,NaN,2685,Europe/Andorra,2014-11-05
3,3017832,Pic de les Abelletes,Pic de les Abelletes,"Pic de la Font-Negre,Pic de la Font-Nègre,Pic ...",42.52535,1.73343,T,PK,AD,FR,A9,66,663.0,66146.0,0,NaN,2411,Europe/Andorra,2014-11-05
4,3017833,Estany de les Abelletes,Estany de les Abelletes,"Estany de les Abelletes,Etang de Font-Negre,Ét...",42.52915,1.73362,H,LK,AD,FR,A9,NaN,NaN,NaN,0,NaN,2260,Europe/Andorra,2014-11-05
